In [ ]:
text_templates = [
    'a bad photo of a {}.',
    'a photo of many {}.',
    'a sculpture of a {}.',
    'a photo of the hard to see {}.',
    'a low resolution photo of the {}.',
    'a rendering of a {}.',
    'graffiti of a {}.',
    'a bad photo of the {}.',
    'a cropped photo of the {}.',
    'a tattoo of a {}.',
    'the embroidered {}.',
    'a photo of a hard to see {}.',
    'a bright photo of a {}.',
    'a photo of a clean {}.',
    'a photo of a dirty {}.',
    'a dark photo of the {}.',
    'a drawing of a {}.',
    'a photo of my {}.',
    'the plastic {}.',
    'a photo of the cool {}.',
    'a close-up photo of a {}.',
    'a black and white photo of the {}.',
    'a painting of the {}.',
    'a painting of a {}.',
    'a pixelated photo of the {}.',
    'a sculpture of the {}.',
    'a bright photo of the {}.',
    'a cropped photo of a {}.',
    'a plastic {}.',
    'a photo of the dirty {}.',
    'a jpeg corrupted photo of a {}.',
    'a blurry photo of the {}.',
    'a photo of the {}.',
    'a good photo of the {}.',
    'a rendering of the {}.',
    'a {} in a video game.',
    'a photo of one {}.',
    'a doodle of a {}.',
    'a close-up photo of the {}.',
    'a photo of a {}.',
    'the origami {}.',
    'the {} in a video game.',
    'a sketch of a {}.',
    'a doodle of the {}.',
    'a origami {}.',
    'a low resolution photo of a {}.',
    'the toy {}.',
    'a rendition of the {}.',
    'a photo of the clean {}.',
    'a photo of a large {}.',
    'a rendition of a {}.',
    'a photo of a nice {}.',
    'a photo of a weird {}.',
    'a blurry photo of a {}.',
    'a cartoon {}.',
    'art of a {}.',
    'a sketch of the {}.',
    'a embroidered {}.',
    'a pixelated photo of a {}.',
    'itap of the {}.',
    'a jpeg corrupted photo of the {}.',
    'a good photo of a {}.',
    'a plushie {}.',
    'a photo of the nice {}.',
    'a photo of the small {}.',
    'a photo of the weird {}.',
    'the cartoon {}.',
    'art of the {}.',
    'a drawing of the {}.',
    'a photo of the large {}.',
    'a black and white photo of a {}.',
    'the plushie {}.',
    'a dark photo of a {}.',
    'itap of a {}.',
    'graffiti of the {}.',
    'a toy {}.',
    'itap of my {}.',
    'a photo of a cool {}.',
    'a photo of a small {}.',
    'a tattoo of the {}.',
]


## reprogram

In [ ]:
import torch
model = torch.load('./trained_model/reprogram_trained.pth')

In [ ]:
import pandas as pd

df = pd.read_csv('./dataset/audio/audiocaps/train.csv')

text_list = list(df['caption'])

audio_paths=["./dataset/audio/esc50/ESC-50-master/audio/1-17150-A-12.wav"]

In [ ]:

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
from utils import *

audio_target = load_n_transform_audio_data(audio_paths,device)
all_emb = []
with torch.no_grad():
    tmp_audio_features = model.forward({'audio': audio_target.to(device)})
    all_emb.append(tmp_audio_features['audio'])
test_audio_features = torch.concat(all_emb)
audio_features_norm = test_audio_features / test_audio_features.norm(dim=-1, keepdim=True)


In [ ]:
from utils import *
bpe_path = './ImageBind/imagebind/bpe/bpe_simple_vocab_16e6.txt.gz'
text_prep = load_n_transform_text_data(text_list,bpe_path,device)

all_emb = []
for i in range(text_prep.shape[0]):
    target = text_prep[i:i+10]
    with torch.no_grad():
        tmp_text_features = model.forward({'text': target.to(device)})
        all_emb.append(tmp_text_features['text'])
test_text_features = torch.concat(all_emb)
text_features_norm = test_text_features / test_text_features.norm(dim=-1, keepdim=True)


In [ ]:
test_text_features.shape

In [ ]:
res = torch.softmax(audio_features_norm @ text_features_norm.T, dim=-1)

sorted_tensor, indices = torch.sort(res, descending=True)
np.array(text_list)[indices[:,0:5].cpu()]

In [ ]:

from preprocessing.sun_rgb_d import *

root = './dataset/depth/sun_d/SUNRGBD'

gt_classes = get_gt(root)
test_depth_path, test_image_path, test_target_list = get_data(root, gt_classes, 'alltest')


In [ ]:
from utils import *
from preprocessing.preprocessing_utils import *
depth_preprocess = get_depth_preprocess()

depth_prep = get_preprocessed_depth_data(test_depth_path,depth_preprocess)
print('done')
all_emb = []
for i in range(depth_prep.shape[0]):
    target = depth_prep[i:i+10]
    with torch.no_grad():
        tmp_depth_features = model.forward({'depth': target.to(device)})
        all_emb.append(tmp_depth_features['depth'])
test_depth_features = torch.concat(all_emb)
depth_features_norm = test_depth_features / test_depth_features.norm(dim=-1, keepdim=True)


In [ ]:
from utils import *
img_name_list = os.listdir('./dataset/image/')
img_list = [os.path.join('./dataset/image/', name) for name in img_name_list]

image_prep = load_n_transform_image_data(img_list,device)
print('done')

all_emb = []
for i in range(image_prep.shape[0]):
    target = image_prep[i:i+1]
    with torch.no_grad():
        tmp_image_features = model.forward({'image': target.to(device)})
        all_emb.append(tmp_image_features['image'])
test_image_features = torch.concat(all_emb)
image_features_norm = test_image_features / test_image_features.norm(dim=-1, keepdim=True)


In [ ]:
res = torch.softmax(audio_features_norm @ depth_features_norm.T, dim=-1)

sorted_tensor, indices = torch.sort(res, descending=True)
np.array(test_depth_path)[indices[:,0:5].cpu()]